In [1]:
import gzip

In [2]:
N8_file = '../../../bsa_by_rnaseq/04_snp_calling/05_VariantFiltering/ref_snps_intersect/N8Ref_only_chr_only_filtered.vcf.gz'
sampleList = ['her14He','her14Ma','her19He','her19Ma']
outfile = '../../../bsa_by_rnaseq/05_mapping/allele_readdepth.txt'

In [3]:
# store N8 snp calls

snpDict = {}

fi = gzip.open(N8_file, 'rt')

for line in fi:
    #print(line)
    
    if line[0] == '#': 
        continue
        
    col = line.rstrip().split('\t')
    chrom = col[0]
    pos = col[1]
    ref = col[3]
    alt = col[4]
    
    snpDict[(chrom, pos)] = [ref, alt]
    #print(chrom, pos, ref, alt)

fi.close()

In [4]:
len(snpDict)

46732

In [5]:
sampleDict = {}

for sample in sampleList:
    
    sample_file = '../../../bsa_by_rnaseq/05_mapping/' + sample + '_HaplotypeCaller_all_sites.g.vcf.gz'
    sampleDict[sample] = {}
    #!ls {sample_file}
    
    fi = gzip.open(sample_file, 'rt')

    for line in fi:
        #print(line)

        if line[0] == '#': 
            continue

        col = line.rstrip().split('\t')
        chrom = col[0]
        pos = col[1]
        ref = col[3]
        alt = col[4]

        genotype = col[9].split(':')[0]
        allele_depth = col[9].split(':')[1]

        if (chrom, pos) in snpDict:

            ref_depth = 0
            alt_dept = 0

            if genotype == '0/0':
                if ref == snpDict[(chrom, pos)][0] and alt == '.':
                    #print(chrom, pos, ref, alt, genotype, allele_depth)
                    ref_depth = int(allele_depth)
                else:
                    #print(chrom, pos, ref, alt, genotype, allele_depth)
                    continue

            elif genotype == '0/1' or genotype == '1/1':
                if ref == snpDict[(chrom, pos)][0] and alt == snpDict[(chrom, pos)][1]:
                    #print(chrom, pos, ref, alt, genotype, allele_depth)
                    ref_depth = int(allele_depth.split(',')[0])
                    alt_depth = int(allele_depth.split(',')[1])
                else:
                    #print(line)
                    continue

            else:
                #print(chrom, pos, ref, alt, genotype, allele_depth)
                continue

            total_depth = ref_depth + alt_depth
            if total_depth >= 1:
            #if ref_depth > 0 or alt_depth > 0:
                sampleDict[sample][(chrom, pos)] = [ref_depth, alt_depth]
            #print(chrom, pos, ref, alt, genotype, allele_depth, ref_depth, alt_depth)

    fi.close()

In [6]:
# number of SNVs per sample if total_depth >=1
for sample in sampleDict:
    print(sample, len(sampleDict[sample]))

her14He 40380
her14Ma 44503
her19He 38389
her19Ma 42246


In [7]:
fo = open(outfile, 'w')

fo.write('#chrom\tpos\tref\talt')

for sample in sampleDict:
    fo.write('\t' + sample + '_ref_depth\t' + sample + '_alt_depth')
fo.write('\n')

for snpid in snpDict:
    fo.write(snpid[0] + '\t' + snpid[1] + '\t' + snpDict[snpid][0] + '\t' + snpDict[snpid][1])

    for sample in sampleDict:
        if snpid in sampleDict[sample]:
            fo.write('\t' + str(sampleDict[sample][snpid][0]) + '\t' + str(sampleDict[sample][snpid][1]))
        else:
            fo.write('\t0\t0')
    fo.write('\n')
    
fo.close()